In [28]:
import pandas as pd
import numpy as np

In [31]:
import sklearn.model_selection as ModelSelection
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

In [3]:
df = pd.read_csv('Avoin_data_eduskuntavaalit_2019_valintatiedot.csv', delimiter=',')

In [9]:
X_cat = df.loc[:, 'Suomen pitää olla edelläkävijä ilmastonmuutoksen vastaisessa taistelussa, vaikka se aiheuttaisi suomalaisille kustannuksia.':'On oikein nähdä vaivaa sen eteen, ettei vahingossakaan loukkaa toista.'].replace('-', np.nan).astype('category')

In [11]:
X_dum = pd.get_dummies(df.loc[:, 'Suomen pitää olla edelläkävijä ilmastonmuutoksen vastaisessa taistelussa, vaikka se aiheuttaisi suomalaisille kustannuksia.':'On oikein nähdä vaivaa sen eteen, ettei vahingossakaan loukkaa toista.'].replace('-', np.nan))

In [14]:
y = df.puolue

In [16]:
X_train, X_test, y_train, y_test = ModelSelection.train_test_split(X_cat, y, test_size=.2, random_state=42)

In [51]:
pipe_svc_cat = pipe2 = Pipeline([
    # Yeah not too smart way to handle missing data at this instance but has to do for now
    ('imp', SimpleImputer(missing_values=np.nan, strategy='mean')), 
    ('scaler', StandardScaler()), 
    ('pca', PCA()), 
    ('svc', SVC())])

In [52]:
param_grid = {
    'svc__gamma':[1e-04, 1e-03, 1e-02],
    'svc__C': [.01, 1, 100, 150, 200, 250],
    'pca__n_components':[25, 20, 15, 10, 5, 2]
    }
gs_svc_cat = GridSearchCV(pipe_svc_cat, param_grid=param_grid, cv=5, scoring='f1_micro')

In [53]:
gs_svc_cat.fit(X_train, y_train)

File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_pca.py", line 376, in fit_transform
    U, S, V = self._fit(X)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_pca.py", line 423, in _fit
    return self._fit_full(X, n_components)
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_pca.py", line 439, in _fit_full
    raise ValueError("n_components=%r must be between 0 and "
ValueError: n_components=40 must be between 0 and min(n_samples, n_features)=29 with svd_solver='full'

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    esti

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imp', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('pca', PCA()), ('svc', SVC())]),
             param_grid={'pca__n_components': [40, 20, 10, 5, 2],
                         'svc__C': [0.01, 1, 100, 250],
                         'svc__gamma': [0.0001, 0.001, 0.01]},
             scoring='f1_micro')

In [54]:
print(f"For training data: \n\tScore: {gs_svc_cat.best_score_} with values {gs_svc_cat.best_params_}")

For training data: 
	Score: 0.5597521587238811 with values {'pca__n_components': 20, 'svc__C': 100, 'svc__gamma': 0.001}
